## Testeo de modelo


In [1]:
import pandas as pd
import joblib
import os

os.chdir('../')

#### Cargar datos de test

In [2]:
df_test = joblib.load("datos/data_test.joblib")

In [5]:
df_test.head()

,motivo,departamento_inmueble,municipio_inmueble,barrio,vias_pavimentadas,sardineles_en_las_vias,andenes_en_las_vias,estrato,topografia_sector,demanda_interes,...,calidad_acabados_banos,estado_acabados_cocina,calidad_acabados_cocina,tipo_garaje,numero_total_de_garajes,total_cupos_parquedaro,tipo_deposito,numero_total_depositos,area_libre,clean_valor_total_avaluo
11508,Empleados,ANTIOQUIA,BELLO,NAVARRA,Si,Si,Si,3.0,Ligera,2.0,...,2.0,3.0,3.0,0.0,0.0,0.0,0,0.0,Si,119702560.0
2157,Crédito hipotecario de vivienda,MAGDALENA,SANTA MARTA,San Jose,Si,Si,Si,3.0,Plano,2.0,...,2.0,3.0,2.0,0.0,0.0,0.0,0,0.0,No,163886500.0
4605,Leasing Habitacional,"BOGOTÁ, D. C.","BOGOTÁ, D. C.",Prado Pinzon,Si,Si,Si,4.0,Plano,2.0,...,2.0,3.0,3.0,2.0,1.0,1.0,Privado,1.0,No,400800000.0
11065,Crédito hipotecario de vivienda,ATLÁNTICO,SOLEDAD,ALTO PRADO,Si,Si,Si,6.0,Ligera,2.0,...,2.0,3.0,2.0,2.0,1.0,2.0,0,0.0,No,315315280.0
10638,Crédito hipotecario de vivienda,QUINDÍO,ARMENIA,Oro Negro,Si,Si,Si,4.0,Ligera,2.0,...,2.0,3.0,3.0,2.0,1.0,1.0,Privado,1.0,No,212418000.0


In [6]:
df_test.shape

(2539, 90)

#### Cargar encoder, selector de variables y mejor modelo

In [25]:
import joblib
from pycaret.regression import load_model 

fwiz = joblib.load("modelos/fwiz.joblib")
cat_econder = joblib.load("modelos/cat_econder.joblib")
vars_dict = joblib.load("modelos/vars_dict.joblib")
model = load_model(r"modelos\best_model-pipeline") 

Transformation Pipeline and Model Successfully Loaded


#### Crear pipeline de predicción

In [26]:
from sklearn.pipeline import make_pipeline

pred_pipeline = make_pipeline(cat_econder, fwiz, model)

Exportar pipeline

In [35]:
joblib.dump(pred_pipeline, "modelos/pred_pipeline.joblib")

['modelos/pred_pipeline.joblib']

#### Cargar mejor modelo y generar predicciones

In [36]:
y_pred = pred_pipeline.predict(df_test.drop(columns=["clean_valor_total_avaluo"]))
y_true = X_test_selected["clean_valor_total_avaluo"]

In [37]:
from sklearn.metrics import mean_absolute_percentage_error
print("MAPE: ",mean_absolute_percentage_error(y_true, y_pred))

MAPE:  0.2835571357409368


### Importancia de variables

In [38]:
importances = model.steps[2][1].feature_importances_
feature_names = X_test_selected.drop(columns=["clean_valor_total_avaluo"]).columns
forest_importances = pd.DataFrame(importances, index=feature_names, columns=['importance']).reset_index()
forest_importances['orig_var'] = forest_importances['index'].apply(lambda v: vars_dict[v])
forest_importances.groupby(['orig_var'])['importance'].max().sort_values(ascending=False).head(30).index

Index(['barrio', 'area_valorada', 'vetustez', 'ocupante',
       'total_cupos_parquedaro', 'municipio_inmueble', 'estrato', 'cocina',
       'clase_inmueble', 'numero_piso', 'estructura_reforzada',
       'departamento_inmueble', 'tipo_garaje', 'detalle_material',
       'tipo_inmueble', 'closet', 'bano_privado', 'balcon',
       'calidad_acabados_madera', 'tanque_de_agua', 'terraza',
       'calidad_acabados_metal', 'demanda_interes', 'telefono_en_el_predio',
       'gas_en_el_predio', 'calidad_acabados_cocina', 'numero_de_edificios',
       'calidad_acabados_techos', 'tipo_deposito', 'motivo'],
      dtype='object', name='orig_var')

### Predicción inmueble promedio

In [39]:
importances = model.steps[2][1].feature_importances_
feature_names = X_test_selected.drop(columns=["clean_valor_total_avaluo"]).columns
forest_importances = pd.DataFrame(importances, index=feature_names, columns=['importance']).reset_index()
forest_importances['orig_var'] = forest_importances['index'].apply(lambda v: vars_dict[v])
forest_importances.groupby(['orig_var'])['importance'].max().sort_values(ascending=False).head(30).index

Index(['barrio', 'area_valorada', 'vetustez', 'ocupante',
       'total_cupos_parquedaro', 'municipio_inmueble', 'estrato', 'cocina',
       'clase_inmueble', 'numero_piso', 'estructura_reforzada',
       'departamento_inmueble', 'tipo_garaje', 'detalle_material',
       'tipo_inmueble', 'closet', 'bano_privado', 'balcon',
       'calidad_acabados_madera', 'tanque_de_agua', 'terraza',
       'calidad_acabados_metal', 'demanda_interes', 'telefono_en_el_predio',
       'gas_en_el_predio', 'calidad_acabados_cocina', 'numero_de_edificios',
       'calidad_acabados_techos', 'tipo_deposito', 'motivo'],
      dtype='object', name='orig_var')

In [40]:
import pickle
datos = pickle.load(open("datos/datos_tablero.pkl","rb")).drop(columns=["clean_valor_total_avaluo","direccion_inmueble_informe","Longitud", "Latitud"])
inmueble_promedio = {'departamento_inmueble':datos['departamento_inmueble'].mode().values[0]}
inmueble_promedio['municipio_inmueble']=datos.query("departamento_inmueble=='{}'".format(inmueble_promedio['departamento_inmueble']))['municipio_inmueble'].mode().values[0]
inmueble_promedio['barrio']=datos.query("municipio_inmueble=='{}'".format(inmueble_promedio['municipio_inmueble']))['barrio'].mode().values[0]

for c in datos.columns:
    if c not in inmueble_promedio.keys():
        if datos[c].dtype == "O":
            inmueble_promedio[c] = datos[c].mode().values[0]
        else:
            inmueble_promedio[c] = datos[c].median()

In [41]:
inmueble_promedio_t = cat_econder.transform(pd.DataFrame.from_dict(inmueble_promedio, orient='index').T)
inmueble_promedio_selected = fwiz.transform(inmueble_promedio_t)

In [42]:
model.predict(inmueble_promedio_selected)[0]

293799152.0

In [43]:
pickle.dump(inmueble_promedio, open("modelos\inmueble_promedio.pkl","wb"))